<a href="https://colab.research.google.com/github/giorgioiacono001-bit/Financial-Markets-Radar/blob/main/Script_completo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance pandas openpyxl


In [25]:
import pandas as pd
import yfinance as yf
import time
import numpy as np
from google.colab import files
import matplotlib.pyplot as plt
import plotly.express as px

In [3]:
from google.colab import files

uploaded = files.upload()


Saving Aziende.xlsx to Aziende.xlsx


In [4]:
df = pd.read_excel("Aziende.xlsx")

# normalizziamo i nomi delle colonne
df.columns = df.columns.str.strip().str.lower()

df.head()

,titolo,ticker,cap_category
0,A2A,A2A.MI,Large
1,AMPLIFON,AMP.MI,Large
2,AZIMUT,AZM.MI,Large
3,BANCA MEDIOLANUM,BMED.MI,Large
4,BANCA MONTE PASCHI SIENA,BMPS.MI,Large


In [17]:
# Normalizziamo il nome della colonna
df.columns = df.columns.str.strip().str.lower()
df.columns

Index(['titolo', 'ticker', 'cap_category'], dtype='object')

In [18]:
#SCARICHIAMO I PREZZI GIORNALIERI
start_date = "2025-01-01"
all_prices_list = []

for ticker in df['ticker'].dropna().unique():
    try:
        print(f"Scarico {ticker}...")
        data = yf.download(ticker, start=start_date, progress=False, auto_adjust=False)
        if data.empty:
            continue

        # Flatten MultiIndex columns
        if isinstance(data.columns, pd.MultiIndex):
            data.columns = ["_".join(col).strip('_') for col in data.columns.values]
        data.columns = ["_".join(col.split('_')[:-1]).strip('_') for col in data.columns.values]

        data.reset_index(inplace=True)
        data.columns = data.columns.str.lower().str.replace(' ', '_')

        # Add ticker
        data["ticker"] = ticker

        # Se cap_category esiste nel df originale, aggiungila
        if 'cap_category' in df.columns:
            # Trova valore corrispondente al ticker
            cap = df.loc[df['ticker'] == ticker, 'cap_category'].iloc[0]
            data['cap_category'] = cap

        # Seleziona solo le colonne come prima, più cap_category
        cols_to_keep = ["ticker", "date", "adj_close", "close", "volume"]
        if 'cap_category' in data.columns:
            cols_to_keep.append("cap_category")
        data = data[cols_to_keep]

        all_prices_list.append(data)
        time.sleep(0.1)

    except Exception as e:
        print(f"Errore su {ticker}: {e}")

# Concatenate
if all_prices_list:
    all_prices = pd.concat(all_prices_list, axis=0, ignore_index=True)
else:
    all_prices = pd.DataFrame(columns=["ticker", "date", "adj_close", "close", "volume", "cap_category"])


Scarico A2A.MI...
Scarico AMP.MI...
Scarico AZM.MI...
Scarico BMED.MI...
Scarico BMPS.MI...
Scarico BAMI.MI...
Scarico BPSO.MI...
Scarico BC.MI...
Scarico BZU.MI...
Scarico CPR.MI...
Scarico DIA.MI...
Scarico ENEL.MI...
Scarico ENI.MI...
Scarico RACE.MI...
Scarico FCT.MI...
Scarico FBK.MI...
Scarico G.MI...
Scarico ISP.MI...
Scarico INW.MI...
Scarico IG.MI...
Scarico LDO.MI...
Scarico MB.MI...
Scarico MONC.MI...
Scarico NEXI.MI...
Scarico PRY.MI...
Scarico REC.MI...
Scarico SPM.MI...
Scarico SRG.MI...
Scarico STLAM.MI...
Scarico STMMI.MI...
Scarico TIT.MI...
Scarico TEN.MI...
Scarico TRN.MI...
Scarico UCG.MI...
Scarico UNI.MI...
Scarico ACE.MI...
Scarico ARN.MI...
Scarico ANIM.MI...
Scarico ARIS.MI...
Scarico ASC.MI...
Scarico AVIO.MI...
Scarico IF.MI...
Scarico BDB.MI...
Scarico BFF.MI...
Scarico BRE.MI...
Scarico CED.MI...
Scarico CRL.MI...
Scarico CMB.MI...
Scarico COM.MI...
Scarico DIS.MI...
Scarico DLG.MI...
Scarico ELN.MI...
Scarico ENAV.MI...
Scarico ERG.MI...
Scarico FM.MI...
S

In [19]:
prices_df = all_prices.copy()
display(prices_df.head())

,ticker,date,adj_close,close,volume,cap_category
0,A2A.MI,2025-01-02,2.092037,2.187,4848444,Large
1,A2A.MI,2025-01-03,2.079602,2.174,3760002,Large
2,A2A.MI,2025-01-06,2.062383,2.156,6386942,Large
3,A2A.MI,2025-01-07,2.097776,2.193,9635926,Large
4,A2A.MI,2025-01-08,2.118821,2.215,12934471,Large


In [24]:
#CALCOLO RENDIMENTI
df = all_prices.copy()
# rendimento giornaliero
df["ret_1d"] = (
    df.groupby("ticker")["adj_close"]
    .pct_change(1)
)

# rendimento a 5 giorni
df["ret_5d"] = (
    df.groupby("ticker")["adj_close"]
    .pct_change(5)
)

# rendimento a 10 giorni
df["ret_10d"] = (
    df.groupby("ticker")["adj_close"]
    .pct_change(10)
)

# rendimento a 20 giorni
df["ret_20d"] = (
    df.groupby("ticker")["adj_close"]
    .pct_change(20)
)

df.head(20)

,ticker,date,adj_close,close,volume,cap_category,ret_1d,ret_5d,ret_10d,ret_20d
0,A2A.MI,2025-01-02,2.092037,2.187,4848444,Large,NaN,NaN,NaN,NaN
1,A2A.MI,2025-01-03,2.079602,2.174,3760002,Large,-0.005944,NaN,NaN,NaN
2,A2A.MI,2025-01-06,2.062383,2.156,6386942,Large,-0.008280,NaN,NaN,NaN
3,A2A.MI,2025-01-07,2.097776,2.193,9635926,Large,0.017161,NaN,NaN,NaN
4,A2A.MI,2025-01-08,2.118821,2.215,12934471,Large,0.010032,NaN,NaN,NaN
5,A2A.MI,2025-01-09,2.155171,2.253,8658172,Large,0.017156,0.030178,NaN,NaN
6,A2A.MI,2025-01-10,2.086298,2.181,17193165,Large,-0.031957,0.003220,NaN,NaN
7,A2A.MI,2025-01-13,2.081515,2.176,6616831,Large,-0.002293,0.009276,NaN,NaN
8,A2A.MI,2025-01-14,2.115951,2.212,11613445,Large,0.016544,0.008664,NaN,NaN
9,A2A.MI,2025-01-15,2.163780,2.262,8204812,Large,0.022604,0.021219,NaN,NaN


In [28]:
#CALCOLO MOMENTUM SCORE
#La velocità è il rendimento a 20 giorni e l'accelerazione è il rapporto tra rendimenti 5 giorni e rendimento 20 giorni
df["velocity"] = df["ret_20d"]

df["acceleration"] = np.where(
    df["ret_20d"] != 0,
    df["ret_5d"] / df["ret_20d"],
    np.nan
)

#Persistenza, ossia qualità del movimento: quanti giorni sono positivi negli ultimi N?
df["pos_day"] = (df["ret_1d"] > 0).astype(int)

df["persistence_20d"] = (
    df.groupby("ticker")["pos_day"]
    .rolling(20)
    .sum()
    .reset_index(level=0, drop=True)
)

#Trend: prezzo sopra SMA50 + direzione positiva: se 1 momentum a favore di trend; se 0 controtrend
df["sma_50"] = (
    df.groupby("ticker")["adj_close"]
    .rolling(50)
    .mean()
    .reset_index(level=0, drop=True)
)

df["trend_flag"] = np.where(
    (df["adj_close"] > df["sma_50"]) &
    (df["sma_50"] > df.groupby("ticker")["sma_50"].shift(5)),
    1,
    0
)


#Anomalia: quanto il rendimento attuale è fuoriscala rispetto al titolo stesso: Z>2 movimento anomalo; Z>3 evento raro
rolling_mean = (
    df.groupby("ticker")["ret_1d"]
    .rolling(60)
    .mean()
    .reset_index(level=0, drop=True)
)

rolling_std = (
    df.groupby("ticker")["ret_1d"]
    .rolling(60)
    .std()
    .reset_index(level=0, drop=True)
)

df["z_score"] = (df["ret_1d"] - rolling_mean) / rolling_std


#Volumi: volume relativo rispetto alla media dei volumi: <1 scarso interesse; 2 partecipazione forte
df["vol_avg_20"] = (
    df.groupby("ticker")["volume"]
    .rolling(20)
    .mean()
    .reset_index(level=0, drop=True)
)

df["rel_volume"] = df["volume"] / df["vol_avg_20"]

#Momentum relativo: non abbiamo ticker FTSEMIB quindi assumiamo come rendimento di mercato la media dei rendimenti
# rendimento medio di mercato (20 giorni) per ogni data
market_ret_20d = (
    df.groupby("date")["ret_20d"]
    .median()   # usa median, più robusta della mean
    .rename("market_ret_20d")
)

df = df.merge(market_ret_20d, on="date", how="left")

# momentum relativo
df["relative_momentum"] = df["ret_20d"] - df["market_ret_20d"]

#se 0 titolo sovraperforma; se <0 sottoperforma




In [29]:
#Normalizzazione

def zscore_cross(x):
    return (x - x.mean()) / x.std()

features = [
    "velocity",
    "acceleration",
    "persistence_20d",
    "z_score",
    "rel_volume",
    "relative_momentum"
]

for f in features:
    df[f + "_z"] = df.groupby("date")[f].transform(zscore_cross)

In [32]:
#Valore momentum
df["momentum_score"] = (
    0.25 * df["velocity_z"] +
    0.20 * df["acceleration_z"] +
    0.15 * df["persistence_20d_z"] +
    0.15 * df["z_score_z"] +
    0.15 * df["rel_volume_z"] +
    0.10 * df["relative_momentum_z"]
)

df["momentum_change"] = df.groupby("ticker")["momentum_score"].diff(5)

In [33]:
df["volatility_20d"] = (
    df.groupby("ticker")["ret_1d"]
    .rolling(20)
    .std()
    .reset_index(level=0, drop=True)
)

df["volatility_20d_z"] =(
    df.groupby("date")["volatility_20d"]
    .transform(zscore_cross)
)

In [20]:
# This cell is no longer needed as column standardization and concatenation are handled in cell 3wbS83TBtx0v.

In [35]:
output_file = "Financial Markets Radar.xlsx"
prices_df.to_excel(output_file, index=False)
files.download(output_file)



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [42]:
#GRAFICI
last_date = df["date"].max()
df_last = df[df["date"] == last_date]
#FIGURA
fig = px.scatter(
    df_last,
    x="momentum_score",
    y="volatility_20d_z",
    hover_name="ticker",
    title=f"Momentum vs Volatilità — {last_date}"
)
fig.show()

#FIGURA 1
fig1 = px.scatter(
    df_last,
    x="momentum_score",
    y="momentum_change",
    hover_name="ticker",
    title=f"Momentum vs Momentum Change — {last_date}"
)
fig1.show()

#FIGURA 2
df["momentum_score_5d_avg"] = (
    df.groupby("ticker")["momentum_score"]
    .rolling(5)
    .mean()
    .reset_index(level=0, drop=True)
)

df["momentum_change_5d_avg"] = (
    df.groupby("ticker")["momentum_change"]
    .rolling(5)
    .mean()
    .reset_index(level=0, drop=True)
)

# --- FIGURA 2: ultimo giorno rolling avg ---
df_last_avg = df[df["date"] == last_date].copy()
df_last_avg = df_last_avg.dropna(subset=["momentum_score_5d_avg", "momentum_change_5d_avg"])

fig2 = px.scatter(
    df_last_avg,
    x="momentum_score_5d_avg",
    y="momentum_change_5d_avg",
    hover_name="ticker",
    title=f"5-day Momentum Score Avg vs. 5-day Momentum Change Avg — {last_date}"
)
fig2.show()